# Log into an Existing FABRIC node

This notebook will show you how to get an existing slice, get a node from the slice, get the IP and make the ssh command that you can use to access the node.


## Configure the Environment

In [ ]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRCI_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

#### Read the SSH Key File

In [ ]:
slice_private_key_file=os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']
slice_public_key_file=os.environ['FABRCI_SLICE_PUBLIC_KEY_FILE']

slice_public_key = None
with open (slice_public_key_file, "r") as myfile:
    slice_public_key=myfile.read().strip()
    

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

#### Create the FABRIC Proxies

The FABRIC API is used via proxy objects that manage connections to the control framework.  

In [ ]:
print(f"FABRIC_ORCHESTRATOR_HOST: {os.environ['FABRIC_ORCHESTRATOR_HOST']}")
print(f"FABRIC_CREDMGR_HOST:      {os.environ['FABRIC_CREDMGR_HOST']}")
print(f"FABRIC_TOKEN_LOCATION:    {os.environ['FABRIC_TOKEN_LOCATION']}")


slice_manager = SliceManager(oc_host=os.environ['FABRIC_ORCHESTRATOR_HOST'], 
                             cm_host=os.environ['FABRIC_CREDMGR_HOST'] ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, project_name='all', scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Configure Slice Parameters



In [ ]:
slice_name="MySlice"
node_name="Node1"
username='ubuntu' # or 'centos'

## Query Slices

You can get a list of all your slices from the slice manager. If this is your first slice, it should return only one slice.

Note that the status returned by the call to slices indicates the success or failure of the call to the FABRIC control framework. The status is not the status of the slices.  The status of each slice is included in the list of slices.

In [ ]:
return_status, slices = slice_manager.slices(excludes=[SliceState.Dead])

if return_status == Status.OK:
    for slice in slices:
        print("{}:".format(slice.slice_name))
        print("   ID         : {}".format(slice.slice_id))
        print("   State      : {}".format(slice.slice_state))
        print("   Lease End  : {}".format(slice.lease_end))
        print()
else:
    print(f"Failure: {slices}")

## Get the Slice

You slice is in the list of all your slices. You can loop through the list of slices to get the slice. Python has a standard tool to filter lists. Try using a lambda function to filter out your slice using its name.

In [ ]:
slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]

print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

## Get the Slice Topology

In [ ]:
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

## Get the Node

In [ ]:
node = experiment_topology.nodes[node_name]
management_ip = str(node.get_property(pname='management_ip'))
print("Node:")
print("   Management IP     : {}".format(node.get_property(pname='management_ip')))
print()    

## Accessing the VM

#### Terminal SSH Command

Connecting to your VMs requests you to jump through the bastion host to the VM using the keys configured above. From a terminal you can ssh to the VM using the command generated by the following command.


In [ ]:
print('ssh -i {} -J {}@{} {}@{}'.format(slice_private_key_file,
                                           os.environ['FABRIC_BASTION_USERNAME'],
                                           os.environ['FABRIC_BASTION_HOST'],
                                           username,
                                           node.management_ip))

#### SSH with Python (paramiko)

Most experiments will require automated configuration and execution. You can use the paramiko Python library to execute arbitrary commands on your VMs. 

The following cells use paramiko to execute a "Hello, FABRIC" bash script. The cell uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.


In [ ]:
script= '#!/bin/bash  \n' \
        'echo Hello, FABRIC from node `hostname -s`   \n'

In [ ]:
from ipaddress import ip_address, IPv4Address
def validIPAddress(IP: str) -> str:
    try:
        return "IPv4" if type(ip_address(IP)) is IPv4Address else "IPv6"
    except ValueError:
        return "Invalid"

import paramiko
for node_name, node in experiment_topology.nodes.items():
    try:
        management_ip = str(node.get_property(pname='management_ip'))
        print("Node {0} IP {1}".format(node_name, management_ip))

        key = paramiko.RSAKey.from_private_key_file(slice_private_key_file)

        bastion=paramiko.SSHClient()
        bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        bastion.connect(os.environ['FABRIC_BASTION_HOST'], 
                        username=os.environ['FABRIC_BASTION_USERNAME'], 
                        key_filename=os.environ['FABRIC_BASTION_KEY_LOCATION'])

    
        bastion_transport = bastion.get_transport()
        if validIPAddress(management_ip) == 'IPv4':
            src_addr = (os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'], 22)
        elif validIPAddress(management_ip) == 'IPv6':
            src_addr = (os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'], 22)
        else:
            print('Management IP Invalid: {}'.format(management_ip))

        dest_addr = (management_ip, 22)
        bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


        client = paramiko.SSHClient()
        client.load_system_host_keys()
        client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
        client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

        client.connect(management_ip,username=username,pkey = key, sock=bastion_channel)

        stdin, stdout, stderr = client.exec_command('echo \"' + script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
        print ('')
        print (str(stdout.read(),'utf-8').replace('\\n','\n'))

        client.close()
    except Exception as e:
        print(str(e))

